In [ ]:
# Austin Diamond

## **Problem**

Janet Williams, the membership manager at 24/7 Fitness Center, has collected data from 1,000 past open house attendees (Gym_Data) and 23 new open house attendees (Gym_Score). The data include the age and annual income of the attendee, the average number of hours that the attendee exercises per week, and for the Gym_Data worksheet, whether or not the attendee purchases a membership. Janet is interested in a model that will help her identify attendees who are likely to purchase a membership.

Perform KNN analysis to estimate a classification model and evaluate its performance. Report the relevant performance measures and score new records.

In [ ]:
import pandas as pd

In [ ]:
from google.colab import files
files.upload();

Saving gym_data.csv to gym_data.csv


Creat a feature matrix **X** and a target variable **y**

In [ ]:
 gym_data = pd.read_csv("gym_data.csv")

In [ ]:
 gym_data.head()

,Enroll,Age,Income,Hours
0,1,26,18000,14
1,0,43,13000,9
2,1,55,42000,16
3,1,55,100000,13
4,0,55,13000,12


In [ ]:
 # check for target variable balance
 gym_data.Enroll.value_counts()

0    597
1    403
Name: Enroll, dtype: int64

Create a feature matrix X and a target variable Y

In [ ]:
X = gym_data.drop("Enroll", axis=1)

In [ ]:
y = gym_data["Enroll"]

Partition the data into 70\% training and 30\% test sets


In [ ]:
# import train test function
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [ ]:
len(X_train)

700

In [ ]:
len(X_test)

300

Use StandardScaler and fit the scaler with our training variables

store the fitted scaler in a variable with the label train_scaler.

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
train_scaler = scaler.fit(X_train)

Apply the scaler to the train and testing data using the transform method.

(base it on the training data variables to avoid data leakage)

In [ ]:
X_train_scaled = train_scaler.transform(X_train)
X_test_scaled = train_scaler.transform(X_test)

In [ ]:
X_train

,Age,Income,Hours
541,42,91000,18
440,57,78000,11
482,58,142000,12
422,43,12000,4
778,45,2000,10
...,...,...,...
106,23,52000,10
270,29,47000,10
860,65,114000,9
435,52,69000,3


In [ ]:
X_train_scaled

array([[-1.90959117e-01,  4.87401421e-01,  1.66159354e+00],
       [ 8.86456643e-01,  1.99260517e-01,  1.47683472e-01],
       [ 9.58284361e-01,  1.61780035e+00,  3.63956338e-01],
       ...,
       [ 1.46107838e+00,  9.97189174e-01, -2.84862261e-01],
       [ 5.27318057e-01, -2.21646849e-04, -1.58249946e+00],
       [-9.81064008e-01, -7.75985619e-01,  5.80229204e-01]])

Set up the KNN model

In [ ]:
# import knn algo
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier()

### Hyperparameter Optimizaiton - GridSearch

Create a dictionary of values to evaluate many different values of K in the model (range between 1 and 10 exclusive)

In [ ]:
 paramGrid = {"n_neighbors": range(1, 10 + 1)}

In [ ]:
 # import gridsearch function
 from sklearn.model_selection import GridSearchCV

In [ ]:
grid_knn = GridSearchCV(model, paramGrid, cv = 10, scoring = "accuracy")

use the fit method on the search variable and store the result in grid_knn.

In [ ]:
grid_knn.fit(X_train_scaled, y_train)

GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 16)}, scoring='accuracy')

In [ ]:
 pd.DataFrame(grid_knn.cv_results_).sort_values("rank_test_score")[["param_n_neighbors", "mean_test_score", "rank_test_score"]]

,param_n_neighbors,mean_test_score,rank_test_score
4,5,0.897143,1
8,9,0.897143,1
10,11,0.895714,3
2,3,0.895714,4
12,13,0.895714,4
13,14,0.895714,4
14,15,0.895714,4
6,7,0.894286,8
9,10,0.891429,9
11,12,0.890000,10


In [ ]:
 grid_knn.best_estimator_

KNeighborsClassifier()

best_estimator_ shows the best parameters

In [ ]:
 grid_knn.best_estimator_

KNeighborsClassifier()

In [ ]:
 y_pred = grid_knn.predict(X_test_scaled)

Accuracy score caluclations

In [ ]:
 compare_prediction = pd.DataFrame({"Actual": y_test, "Predicted": y_pred, "compare": y_test == y_pred})

In [ ]:
compare_prediction

,Actual,Predicted,compare
521,0,0,True
737,0,0,True
740,1,1,True
660,0,0,True
411,0,0,True
...,...,...,...
468,1,1,True
935,0,0,True
428,1,0,False
7,0,0,True


In [ ]:
compare_prediction.value_counts()

Actual  Predicted  compare
0       0          True       169
1       1          True       111
0       1          False       11
1       0          False        9
dtype: int64

Classification Report

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.94      0.94       180
           1       0.91      0.93      0.92       120

    accuracy                           0.93       300
   macro avg       0.93      0.93      0.93       300
weighted avg       0.93      0.93      0.93       300

